In [59]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

In [ ]:
cd dataset

Our goal is to link all the data together using the police district number. 

Each row will represent the features of that police district (eg. car crash, crime) in a weekly/monthly basis

## Load related data

In [61]:
with open('Data Open 2023 Additional Data/philadelphia.geojson') as handle:
    philly_geo = json.loads(handle.read())
  
district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

## Police District

In [62]:
df_police_districts = pd.read_csv("Traffic, Investigations _ Other/police_districts.csv")
df_police_districts.head()

,OBJECTID,PERIMETER,DISTRICT_NUM,LOCATION,LAT,LNG,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,343,69282.58846,16,39th St. & Lancaster Ave.,39.96158,-75.19940,SWPD,1.216700e+08,1.927226e+07,27575.07918
1,344,33150.15496,17,20th St. & Federal St.,39.93728,-75.17670,SPD,5.786368e+07,9.154950e+06,13179.95335
2,345,54403.93004,18,55th St. & Pine St.,39.95426,-75.23217,SWPD,9.881929e+07,1.564117e+07,21629.61069
3,346,51597.05160,35,N. Broad St. & Champlost St.,40.04442,-75.14351,NWPD,1.544309e+08,2.450685e+07,20554.86914
4,347,58075.01444,39,22nd St. & Hunting Park Ave.,40.01362,-75.15884,NWPD,1.578976e+08,2.503787e+07,23123.58026


## Crime with district

In [63]:
crime = pd.read_csv("Traffic, Investigations _ Other/crimes.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(crime['lng'], crime['lat'])]
crime_gdf = gpd.GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)
crime_with_district = gpd.sjoin(crime_gdf, district_gdf, how='left', op='within')
crime_with_district = crime_with_district.dropna(subset=["DIST_NUM"])
print(len(crime_with_district))
crime_with_district.head()

1573848


,objectid,psa,dispatch_date,dispatch_time,location_block,text_general_code,lat,lng,fips,geometry,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,218891,2,2022-09-08,15:21:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,None,39.0,None,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
1,218892,2,2022-09-14,17:41:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,None,39.0,None,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
2,218893,2,2022-09-19,15:23:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,None,39.0,None,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
3,218894,2,2022-08-22,09:38:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,None,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793
4,218895,2,2022-08-20,10:31:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,None,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [64]:
# We will keep the date and the types of crime (text_general_code)
crime_useful_feat = crime_with_district[["dispatch_date", "text_general_code", "DIST_NUM"]]
#crime_useful_feat = crime_useful_feat.join(pd.get_dummies(crime_with_district["text_general_code"]))
crime_useful_feat['dispatch_date'] = pd.to_datetime(crime_useful_feat['dispatch_date'])
crime_useful_feat['year_month'] = crime_useful_feat['dispatch_date'].dt.to_period('M')
crime_useful_feat = crime_useful_feat.drop("dispatch_date", axis=1)
crime_crosstab = pd.crosstab(index = [crime_useful_feat['year_month'],crime_useful_feat['DIST_NUM']], columns = crime_useful_feat['text_general_code'])
crime_crosstab.columns = ["crime_"+i for i in crime_crosstab.columns+"_count"]
crime_crosstab['crime_total_count'] = crime_crosstab.sum(axis=1)
crime_crosstab

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


crime_Aggravated Assault Firearm_count  \
year_month DIST_NUM                                           
2012-12    1.0                                            0   
           2.0                                            0   
           3.0                                            0   
           5.0                                            0   
           6.0                                            0   
...                                                     ...   
2022-12    25.0                                          37   
           26.0                                          11   
           35.0                                          19   
           39.0                                          16   
           77.0                                           0   

                     crime_Aggravated Assault No Firearm_count  \
year_month DIST_NUM                                              
2012-12    1.0                                               0   
           2.0                                               0   
           3.0                                               0   
           5.0                                               0   
           6.0                                               0   
...                                                        ...   
2022-12    25.0                                             42   
           26.0                                             14   
           35.0                                             26   
           39.0                                             30   
           77.0                                              0   

                     crime_All Other Offenses_count  crime_Arson_count  \
year_month DIST_NUM                                                      
2012-12    1.0                                    0                  0   
           2.0                                    0                  0   
           3.0                                    0                  0   
           5.0                                    0                  0   
           6.0                                    0                  0   
...                                             ...                ...   
2022-12    25.0                                  60                  4   
           26.0                                  28                  1   
           35.0                                  59                  0   
           39.0                                  46                  0   
           77.0                                   2                  0   

                     crime_Burglary Non-Residential_count  \
year_month DIST_NUM                                         
2012-12    1.0                                          0   
           2.0                                          0   
           3.0                                          0   
           5.0                                          0   
           6.0                                          0   
...                                                   ...   
2022-12    25.0                                         7   
           26.0                                        23   
           35.0                                         2   
           39.0                                        12   
           77.0                                         0   

                     crime_Burglary Residential_count  \
year_month DIST_NUM                                     
2012-12    1.0                                      0   
           2.0                                      0   
           3.0                                      0   
           5.0                                      0   
           6.0                                      0   
...                                               ...   
2022-12    25.0                                     9   
           26.0                                    28   
           35.0

## Crash with District

In [65]:
df_crash = pd.read_csv("Crashes/crash_info_general.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(df_crash['DEC_LONG'], df_crash['DEC_LAT'])]
crash_gdf = gpd.GeoDataFrame(df_crash, crs="EPSG:4326", geometry=geometry)
crash_with_district = gpd.sjoin(crash_gdf, district_gdf, how='left', op='within')
crash_with_district = crash_with_district.dropna(subset=["DIST_NUM"])
print(len(crash_with_district))
crash_with_district.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


131569


,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,None,16.0,None,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,None,15.0,None,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,None,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [66]:
# We will keep the following features
# categorical feat: *_TYPE, ILLUMINATION, MAX_SEVERITY_LEVEL, RELATION_TO_ROAD, ROAD_CONDITION, WEATHER1
# date feat: CRASH_MONTH, CRASH_YEAR, DATE_OF_WEEK

number_of_crashes = crash_with_district.groupby("DIST_NUM").count()["CRN"]
number_of_crashes


DIST_NUM
1.0      3333
2.0      6270
3.0      6475
5.0      1999
6.0      6995
7.0      4272
8.0      7706
9.0      4967
12.0     8005
14.0     5866
15.0    12130
16.0     8841
17.0     2537
18.0     7567
19.0     4887
22.0     6135
24.0     6893
25.0     7485
26.0     5183
35.0     7089
39.0     6890
77.0       44
Name: CRN, dtype: int64

In [67]:
#split crash_with_district into different year_month units
crash_with_district['date_str'] = crash_with_district['CRASH_YEAR'].astype(str) + '-' + crash_with_district['CRASH_MONTH'].astype(str).str.zfill(2)
crash_with_district['year_month'] = pd.to_datetime(crash_with_district['date_str']).dt.to_period('M')
crash_with_district = crash_with_district.drop('date_str', axis=1)
crash_with_district

,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length,year_month
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692,2010-01
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692,2010-01
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,16.0,None,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183,2010-01
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,15.0,None,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281,2010-01
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793,2010-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133008,133009,2022041832,NaN,1,0,0,0,0,0,0,...,19.0,None,19,61st St. & Thompson St.,686-3190,SWPD,1.782215e+08,2.823350e+07,28509.269384,2021-07
133009,133010,2022043790,0.0,1,0,0,0,0,0,0,...,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692,2021-11
133010,133011,2022047157,1000.0,2,0,0,0,0,0,0,...,25.0,None,25,3700 Whittaker Ave,686-3250,EPD,1.179874e+08,1.870673e+07,19809.830073,2021-07
133011,133012,2022055468,NaN,0,0,0,0,0,0,0,...,22.0,None,22,17th St & Montgomery Ave.,686-3220,CPD,1.201916e+08,1.904228e+07,18230.658403,2021-11


In [68]:
crash_with_district["DRIVER_COUNT_16_20YR"]=crash_with_district[["DRIVER_COUNT_16YR","DRIVER_COUNT_17YR","DRIVER_COUNT_18YR","DRIVER_COUNT_19YR","DRIVER_COUNT_20YR"]].sum(axis=1)
crash_with_district["DRIVER_COUNT_16_20YR"]

1         0
2         0
4         0
5         0
6         0
         ..
133008    0
133009    0
133010    0
133011    0
133012    0
Name: DRIVER_COUNT_16_20YR, Length: 131569, dtype: int64

In [69]:
manual_cat_list = ["ILLUMINATION","ROAD_CONDITION", "WEATHER1"]
manual_drop_list = ["RDWY_SURF_TYPE_CD", "WORK_ZONE_TYPE"]
crash_useful_cat_feat_list = [i for i in crash_with_district.columns if ("_TYPE" in i and i not in manual_drop_list) or i in manual_cat_list]
crash_useful_num_feat_list = [i for i in crash_with_district.columns if "COUNT" in i]+["DRIVER_COUNT_16_20YR","DRIVER_COUNT_50_64YR","DRIVER_COUNT_65_74YR","DRIVER_COUNT_75PLUS"]
crash_useful_feat = crash_with_district[crash_useful_cat_feat_list+crash_useful_num_feat_list]
crash_useful_feat[["DIST_NUM","year_month"]]=crash_with_district[["DIST_NUM","year_month"]]
crash_useful_feat

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,COLLISION_TYPE,ILLUMINATION,INTERSECT_TYPE,LOCATION_TYPE,ROAD_CONDITION,TCD_TYPE,WEATHER1,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,...,UNK_INJ_PER_COUNT,VAN_COUNT,VEHICLE_COUNT,DRIVER_COUNT_16_20YR,DRIVER_COUNT_16_20YR,DRIVER_COUNT_50_64YR,DRIVER_COUNT_65_74YR,DRIVER_COUNT_75PLUS,DIST_NUM,year_month
1,4,5.0,0,0,9,0,3,2,0,0,...,0,0,3,0,0,0,0,0,6.0,2010-01
2,7,1.0,0,0,1,0,3,2,0,0,...,0,0,3,0,0,0,0,0,6.0,2010-01
4,5,3.0,0,0,9,0,7,0,0,0,...,0,0,2,0,0,1,0,0,16.0,2010-01
5,7,3.0,3,2,9,0,3,1,0,0,...,0,0,1,0,0,0,0,0,15.0,2010-01
6,7,3.0,0,0,9,0,3,1,0,0,...,0,0,1,0,0,0,0,0,3.0,2010-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133008,4,1.0,0,7,1,0,3,1,0,0,...,0,1,2,0,0,0,0,0,19.0,2021-07
133009,4,8.0,5,2,1,2,3,1,0,0,...,0,0,2,0,0,1,0,1,6.0,2021-11
133010,1,1.0,0,0,1,0,3,2,0,0,...,0,0,3,0,0,0,0,0,25.0,2021-07
133011,4,1.0,1,0,1,3,3,0,0,0,...,0,0,2,0,0,0,0,0,22.0,2021-11


In [70]:
#compute the Traffic Hazard Impact Index
w1=10 #subjectively
w2=5
w3=2
crash_useful_feat["index"]=w1*crash_useful_feat["FATAL_COUNT"]+\
                    w2*crash_useful_feat["SUSP_SERIOUS_INJ_COUNT"]+w3*crash_useful_feat["SUSP_MINOR_INJ_COUNT"]+1
crash_useful_feat = crash_useful_feat.drop(["FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"], axis=1)

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
crash_useful_feat

,COLLISION_TYPE,ILLUMINATION,INTERSECT_TYPE,LOCATION_TYPE,ROAD_CONDITION,TCD_TYPE,WEATHER1,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,...,VAN_COUNT,VEHICLE_COUNT,DRIVER_COUNT_16_20YR,DRIVER_COUNT_16_20YR,DRIVER_COUNT_50_64YR,DRIVER_COUNT_65_74YR,DRIVER_COUNT_75PLUS,DIST_NUM,year_month,index
1,4,5.0,0,0,9,0,3,2,0,0,...,0,3,0,0,0,0,0,6.0,2010-01,3
2,7,1.0,0,0,1,0,3,2,0,0,...,0,3,0,0,0,0,0,6.0,2010-01,1
4,5,3.0,0,0,9,0,7,0,0,0,...,0,2,0,0,1,0,0,16.0,2010-01,1
5,7,3.0,3,2,9,0,3,1,0,0,...,0,1,0,0,0,0,0,15.0,2010-01,1
6,7,3.0,0,0,9,0,3,1,0,0,...,0,1,0,0,0,0,0,3.0,2010-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133008,4,1.0,0,7,1,0,3,1,0,0,...,1,2,0,0,0,0,0,19.0,2021-07,1
133009,4,8.0,5,2,1,2,3,1,0,0,...,0,2,0,0,1,0,1,6.0,2021-11,8
133010,1,1.0,0,0,1,0,3,2,0,0,...,0,3,0,0,0,0,0,25.0,2021-07,1
133011,4,1.0,1,0,1,3,3,0,0,0,...,0,2,0,0,0,0,0,22.0,2021-11,1


## Traffic Stop Data

In [72]:
traffic_stop_with_district = pd.read_csv("Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")
traffic_stop_with_district.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


In [73]:
traffic_stop_with_district['date'] = pd.to_datetime(traffic_stop_with_district['date'])
traffic_stop_with_district['year_month'] = traffic_stop_with_district['date'].dt.to_period('M')

In [74]:
traffic_stop_with_district = traffic_stop_with_district.rename(columns={"district": "DIST_NUM"})
traffic_stop_count = pd.DataFrame(traffic_stop_with_district.groupby(["DIST_NUM", "year_month"]).count()['objectid'])

In [75]:
traffic_stop_count.columns=["traffic_stop_count"]
traffic_stop_count

traffic_stop_count
DIST_NUM year_month                    
1.0      2014-01                    382
         2014-02                   1132
         2014-03                    993
         2014-04                   1485
         2014-05                   1419
...                                 ...
77.0     2017-12                     76
         2018-01                     65
         2018-02                     64
         2018-03                     87
         2018-04                     35

[1144 rows x 1 columns]

## Merge Crime, Crash, Traffic Stop

In [76]:
traffic_stop_count.reset_index()

,DIST_NUM,year_month,traffic_stop_count
0,1.0,2014-01,382
1,1.0,2014-02,1132
2,1.0,2014-03,993
3,1.0,2014-04,1485
4,1.0,2014-05,1419
...,...,...,...
1139,77.0,2017-12,76
1140,77.0,2018-01,65
1141,77.0,2018-02,64
1142,77.0,2018-03,87


In [77]:
df_merged = crash_useful_feat.merge(traffic_stop_count.reset_index(), on=["DIST_NUM", "year_month"], how='left')
df_merged= df_merged.merge(crime_crosstab.reset_index(), on=["DIST_NUM", "year_month"], how='left')

In [80]:
df_merged

,COLLISION_TYPE,ILLUMINATION,INTERSECT_TYPE,LOCATION_TYPE,ROAD_CONDITION,TCD_TYPE,WEATHER1,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,...,crime_Rape_count,crime_Receiving Stolen Property_count,crime_Robbery Firearm_count,crime_Robbery No Firearm_count,crime_Theft from Vehicle_count,crime_Thefts_count,crime_Vagrancy/Loitering_count,crime_Vandalism/Criminal Mischief_count,crime_Weapon Violations_count,crime_total_count
0,4,5.0,0,0,9,0,3,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,1.0,0,0,1,0,3,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,3.0,0,0,9,0,7,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,3.0,3,2,9,0,3,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,3.0,0,0,9,0,3,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131564,4,1.0,0,7,1,0,3,1,0,0,...,4.0,4.0,3.0,10.0,26.0,95.0,0.0,101.0,28.0,743.0
131565,4,8.0,5,2,1,2,3,1,0,0,...,2.0,0.0,5.0,22.0,67.0,199.0,0.0,48.0,3.0,579.0
131566,1,1.0,0,0,1,0,3,2,0,0,...,6.0,10.0,12.0,21.0,32.0,79.0,0.0,54.0,37.0,668.0
131567,4,1.0,1,0,1,3,3,0,0,0,...,9.0,2.0,23.0,9.0,72.0,103.0,0.0,93.0,12.0,850.0


In [79]:
df_merged.to_csv("df_merged_v3.csv")